In [1]:
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
import requests
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

/home/moogedelic/miniforge3/envs/llm_camp_env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Q1. Getting the embeddings model

In [2]:
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

/home/moogedelic/miniforge3/envs/llm_camp_env/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971214/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Create the embedding for this user question:

In [3]:
user_question = "I just discovered the course. Can I still join it?"
question_vector = model.encode(user_question)
question_vector[0]

0.078222655

What's the first value of the resulting vector? 0.07

Prepare the documents

In [4]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
# Filter the documents
filtered_documents = [doc for doc in documents if doc.get('course') == 'machine-learning-zoomcamp']
len(filtered_documents)

375

Q2. Creating the embeddings

In [5]:
# Create a list to store embeddings
embeddings = []

# Iterate over each filtered document and compute embeddings
for doc in filtered_documents:
    question = doc.get('question', '')
    answer = doc.get('text', '')
    qa_text = f'{question} {answer}'
    embedding = model.encode(qa_text)
    embeddings.append(embedding)

In [6]:
# Convert the list of embeddings to a NumPy array
X = np.array(embeddings)

# Print the shape of the resulting array
print(X.shape)

(375, 768)


What's the shape of X? (375, 768)

Q3. Search

In [7]:
scores = X.dot(question_vector)
scores.max()

0.6506574

What's the highest score in the results? 0.65

Vector search

In [8]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results)[:num_results]
        # Sort the top num_results indices to ensure the results are in order
        idx = idx[np.argsort(-scores[idx])]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(question_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

Q4. Hit-rate for our search engine

In [9]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [10]:
# Define the search function wrapper
def search_function(query):
    return custom_search(query, search_engine, num_results=5)

In [11]:
def custom_search(query, search_engine, num_results=5):
    # Extracting fields from query dictionary
    vector = model.encode(query['question'])  # Encode the query vector
    course = query['course']
    
    # Filtered embeddings are already computed
    filtered_documents = [doc for doc in search_engine.documents if doc.get('course') == course]
    filtered_embeddings = search_engine.embeddings  # Use precomputed embeddings

    # Create a new VectorSearchEngine for the filtered documents and embeddings
    filtered_search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=filtered_embeddings)

    # Perform the search using the filtered search engine
    result_docs = filtered_search_engine.search(vector, num_results=num_results)

    return result_docs


In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [13]:
def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:
                total_score += 1 / (rank + 1)
                break
    return total_score / len(relevance_total)

In [14]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }




Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5

In [15]:
# Evaluate
results = evaluate(ground_truth, search_function)
print(results)


  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8502823315118397}


What did you get? 0.93

Q5. Indexing with Elasticsearch

In [16]:
es_client = Elasticsearch('http://localhost:9200')

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 375,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [22]:
def elastic_search(query, course, num_results=5):
    search_query = {
        "size": num_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "sort": [
            {"_score": {"order": "desc"}}
        ]
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs



In [31]:
def evaluate(ground_truth, course, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q['question'], course)  # Pass query and course to search_function
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [32]:
course ='machine-learning-zoomcamp'
elastic_search(user_question, course)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

What's the ID of the document with the highest score? id': '7842b56a'

Q6. Hit-rate for Elasticsearch

In [33]:
# Evaluate using Elasticsearch
results = evaluate(ground_truth, course, elastic_search)
print(results)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.7491803278688525, 'mrr': 0.6056284153005476}


What's hitrate for our dataset for Elastic? 0.75